# Crear base de datos

In [23]:


import pandas as pd
import json
from tqdm import tqdm


In [24]:


df_estudiantes = pd.read_excel("ESTUDIANTES.xlsx")
df_materias = pd.read_excel("MATERIAS.xlsx")


In [25]:
df_estudiantes

,ID,PERIODO,EDAD,SEXO,SEMESTRE,TIPO_ESTUDIANTE,TIPO_ADMISION,PROGRAMA,PROGRAMA2,ESTRATO,...,ICFES,ICFES MT,ICFES LC,ICFES SC,ICFES CN,ICFES ING,DISCAPACIDAD,GRADUADO,DESERTOR,BECADO
0,409967,202510,17,F,1.0,Estudiante regular,Ordinaria Pregrado,Derecho,NaN,5,...,213.0,46.0,37.0,43.0,41.0,54.0,00- Ninguno,0,0,No becado
1,410620,202510,18,M,1.0,Estudiante regular,Ordinaria Pregrado,Geología,NaN,1,...,257.0,60.0,52.0,43.0,49.0,56.0,00- Ninguno,0,0,No becado
2,410657,202510,17,F,1.0,Estudiante regular,Ordinaria Pregrado,Odontología,NaN,2,...,380.0,77.0,74.0,75.0,70.0,100.0,00- Ninguno,0,0,Institucional
3,415629,202510,18,M,1.0,Estudiante regular,Ordinaria Pregrado,Ingeniería Industrial,NaN,4,...,374.0,78.0,74.0,73.0,73.0,78.0,00- Ninguno,0,0,Institucional
4,415712,202510,18,M,1.0,Estudiante regular,Ordinaria Pregrado,Economía,NaN,4,...,323.0,64.0,65.0,66.0,61.0,72.0,00- Ninguno,0,0,No becado
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10221,370043,202510,24,F,7.0,Estudiante regular,Transferencia Externa,Ingeniería Mecánica,NaN,4,...,NaN,NaN,NaN,NaN,NaN,NaN,00- Ninguno,0,0,No becado
10222,361636,202510,22,M,5.0,Estudiante regular,Ordinaria Pregrado,Psicología,NaN,4,...,NaN,NaN,NaN,NaN,NaN,NaN,00- Ninguno,0,0,No becado
10223,320707,202510,24,F,11.0,Estudiante regular,Ordinaria Pregrado,Medicina,NaN,4,...,NaN,NaN,NaN,NaN,NaN,NaN,00- Ninguno,0,0,No becado
10224,322728,202510,22,M,9.0,Estudiante regular,Ordinaria Pregrado,Administración de Empresas,NaN,6,...,NaN,NaN,NaN,NaN,NaN,NaN,00- Ninguno,1,0,No becado


In [26]:


df_estudiantes.columns = df_estudiantes.columns.str.strip()
df_materias.columns = df_materias.columns.str.strip()


In [27]:


documentos_json = []
estudiantes_procesados = 0
errores = 0

In [28]:

# Iterar sobre cada estudiante
for indice, estudiante in tqdm(df_estudiantes.iterrows(), total=len(df_estudiantes), desc="Procesando estudiantes"):
    
    # Extraer ID del estudiante
    estudiante_id = str(estudiante['ID'])
    
   
    
    # Filtrar materias que pertenecen a este estudiante
    materias_del_estudiante = df_materias[df_materias['ID'] == estudiante['ID']]
    
  
    
    # Lista para guardar materias en formato JSON
    materias_array = []
    
    # Variables para métricas
    materias_perdidas_total = 0
    materias_perdidas_por_categoria = {}  # Diccionario para contar por categoría
    codigos_materias_vistas = {}  # Para detectar repeticiones
    materias_repetidas_count = 0
    
    # Procesar cada materia
    for idx_materia, materia in materias_del_estudiante.iterrows():
        
        # Extraer datos de la materia
        nota = materia['NOTA']
        codigo = materia['CODIGO MAERIA']
        nombre_materia = materia['MATERIA']
        categoria = materia['CATEGORIA MATERIA']
        periodo = materia['PERIODO']
        retirada = materia['Retirada']
        
        # Convertir nota a float (si existe)
        if pd.notna(nota):
            nota_numerica = float(nota)
        else:
            nota_numerica = None
        
        # Convertir retirada a entero
        if pd.notna(retirada):
            retirada_valor = int(retirada)
        else:
            retirada_valor = 0
        
        # Crear objeto de materia para el JSON
        materia_objeto = {
            "materia": nombre_materia,
            "codigo_materia": codigo,
            "categoria": categoria,
            "periodo": int(periodo),
            "nota": nota_numerica,
            "retirada": retirada_valor
        }
        
        # Agregar a la lista
        materias_array.append(materia_objeto)
        
        # Lógica de materias perdidas:
        # 1. Retirada: retirada == 1 (nota vacía o 0)
        # 2. Perdida: nota entre 0 y 3 (exclusivo) y NO retirada
        
        materia_perdida = False
        
        if retirada_valor == 1:
            # Materia retirada (sin importar la nota)
            materia_perdida = True
        elif nota_numerica is not None and 0 < nota_numerica < 3.0:
            # Materia perdida (nota mayor a 0 y menor a 3)
            materia_perdida = True
        
        if materia_perdida:
            materias_perdidas_total += 1
            
            # Contar por categoría
            if categoria in materias_perdidas_por_categoria:
                materias_perdidas_por_categoria[categoria] += 1
            else:
                materias_perdidas_por_categoria[categoria] = 1
        
        # Detectar materias repetidas (mismo código más de una vez)
        if codigo in codigos_materias_vistas:
            codigos_materias_vistas[codigo] += 1
        else:
            codigos_materias_vistas[codigo] = 1
    
    # Contar cuántos códigos se repitieron
    for codigo, veces in codigos_materias_vistas.items():
        if veces > 1:
            materias_repetidas_count += 1
    

    
    # Obtener solo las notas válidas (no None y no retiradas)
    notas_validas = []
    for mat in materias_array:
        if mat['nota'] is not None and mat['retirada'] == 0:
            notas_validas.append(mat['nota'])
    
    # Calcular promedio
    if len(notas_validas) > 0:
        promedio_acumulado = sum(notas_validas) / len(notas_validas)
        promedio_acumulado = round(promedio_acumulado, 2)
    else:
        promedio_acumulado = None
    
    
    # Extraer ciudad y país
    ciudad = str(estudiante['CIUDAD']).strip() if pd.notna(estudiante['CIUDAD']) else ""
    pais = str(estudiante['PAIS']).strip() if pd.notna(estudiante['PAIS']) else ""
    
    # Variables binarias
    es_barranquilla = 1 if ciudad.lower() == 'barranquilla' else 0
    es_colombia = 1 if pais.lower() == 'colombia' else 0
   
    
    documento = {
        "_id": estudiante_id,
        
        "datos_personales": {
            "edad": int(estudiante['EDAD']) if pd.notna(estudiante['EDAD']) else None,
            "genero": str(estudiante['SEXO']).strip() if pd.notna(estudiante['SEXO']) else "",
            "estrato": int(estudiante['ESTRATO']) if pd.notna(estudiante['ESTRATO']) else None,
            "discapacidad": str(estudiante['DISCAPACIDAD']).strip() if pd.notna(estudiante['DISCAPACIDAD']) else ""
        },
        
        "academico": {
            "programa": str(estudiante['PROGRAMA']).strip() if pd.notna(estudiante['PROGRAMA']) else "",
            "programa_secundario": str(estudiante['PROGRAMA2']).strip() if pd.notna(estudiante['PROGRAMA2']) else None,
            "semestre_actual": int(estudiante['SEMESTRE']) if pd.notna(estudiante['SEMESTRE']) else None,
            "tipo_estudiante": str(estudiante['TIPO_ESTUDIANTE']).strip() if pd.notna(estudiante['TIPO_ESTUDIANTE']) else "",
            "tipo_admision": str(estudiante['TIPO_ADMISION']).strip() if pd.notna(estudiante['TIPO_ADMISION']) else "",
            "estado_academico": str(estudiante['ESTADO_ACADEMICO']).strip() if pd.notna(estudiante['ESTADO_ACADEMICO']) else ""
        },
        
        "location": {
            "ciudad": ciudad,
            "departamento": str(estudiante['DEPTO']).strip() if pd.notna(estudiante['DEPTO']) else "",
            "pais": pais,
            "es_barranquilla": es_barranquilla,
            "es_colombia": es_colombia,
            "codigo_dane": str(estudiante['CODIGO_DANE']).strip() if pd.notna(estudiante['CODIGO_DANE']) else None
        },
        
        "colegio": {
            "tipo_colegio": str(estudiante['TIPO COLEGIO']).strip() if pd.notna(estudiante['TIPO COLEGIO']) else None,
            "calendario_colegio": str(estudiante['CALENDARIO COLEGIO']).strip() if pd.notna(estudiante['CALENDARIO COLEGIO']) else None,
            "descripcion_bachillerato": str(estudiante['DESC_BACHILLERATO']).strip() if pd.notna(estudiante['DESC_BACHILLERATO']) else None
        },
        
        "ICFES": {
            "puntaje_total": float(estudiante['ICFES']) if pd.notna(estudiante['ICFES']) else None,
            "matematicas": float(estudiante['ICFES MT']) if pd.notna(estudiante['ICFES MT']) else None,
            "lectura_critica": float(estudiante['ICFES LC']) if pd.notna(estudiante['ICFES LC']) else None,
            "sociales": float(estudiante['ICFES SC']) if pd.notna(estudiante['ICFES SC']) else None,
            "ciencias": float(estudiante['ICFES CN']) if pd.notna(estudiante['ICFES CN']) else None,
            "ingles": float(estudiante['ICFES ING']) if pd.notna(estudiante['ICFES ING']) else None
        },
        
        "metricas_rendimiento": {
            "promedio_acumulado": promedio_acumulado,
            "materias_cursadas_total": len(materias_array),
            "materias_perdidas_total": materias_perdidas_total,
            "materias_perdidas_por_departamento": materias_perdidas_por_categoria,
            "materias_repetidas": materias_repetidas_count
        },
        
        "estado": {
            "becado": str(estudiante['BECADO']).strip() if pd.notna(estudiante['BECADO']) else "",
            "graduado": int(estudiante['GRADUADO']) if pd.notna(estudiante['GRADUADO']) else 0,
            "desertor": int(estudiante['DESERTOR']) if pd.notna(estudiante['DESERTOR']) else 0
        },
        
        "periodo_info": {
            "ultimo_periodo": int(estudiante['PERIODO']) if pd.notna(estudiante['PERIODO']) else None
        },
        
        "materias_cursadas": materias_array
    }
    
    
    
    documentos_json.append(documento)
    estudiantes_procesados += 1
    
       


Procesando estudiantes: 100%|██████████| 10226/10226 [00:10<00:00, 950.61it/s] 


In [29]:

# Guardar con formato bonito (indent=2)
with open('estudiantes_documentos.json', 'w', encoding='utf-8') as archivo:
    json.dump(documentos_json, archivo, ensure_ascii=False, indent=2)


In [30]:




print(json.dumps(documentos_json[0], indent=2, ensure_ascii=False))



{
  "_id": "409967",
  "datos_personales": {
    "edad": 17,
    "genero": "F",
    "estrato": 5,
    "discapacidad": "00- Ninguno"
  },
  "academico": {
    "programa": "Derecho",
    "programa_secundario": null,
    "semestre_actual": 1,
    "tipo_estudiante": "Estudiante regular",
    "tipo_admision": "Ordinaria Pregrado",
    "estado_academico": "Normal"
  },
  "location": {
    "ciudad": "San Andres",
    "departamento": "Archipielago de S.A.y Providen",
    "pais": "Colombia",
    "es_barranquilla": 0,
    "es_colombia": 1,
    "codigo_dane": "388001000673.0"
  },
  "colegio": {
    "tipo_colegio": "PRIVADO",
    "calendario_colegio": "A",
    "descripcion_bachillerato": "Gimnasio Real"
  },
  "ICFES": {
    "puntaje_total": 213.0,
    "matematicas": 46.0,
    "lectura_critica": 37.0,
    "sociales": 43.0,
    "ciencias": 41.0,
    "ingles": 54.0
  },
  "metricas_rendimiento": {
    "promedio_acumulado": 3.47,
    "materias_cursadas_total": 13,
    "materias_perdidas_total": 8,
 

# concetar a azure

In [31]:
#pip install pymongo pandas dnspython

In [32]:
import json
from pymongo import MongoClient
import time

In [33]:

CONNECTION_STRING = "mongodb://proyectoestudiantes:suVKLoKoqjCxdeRnmZH7IICunDq54Ed33zaKcNzPBdxI2PVaohC0veT5diWHmsojaQCW6r2qohC9ACDbu5vPqQ==@proyectoestudiantes.mongo.cosmos.azure.com:10255/?ssl=true&replicaSet=globaldb&retrywrites=false&maxIdleTimeMS=120000&appName=@proyectoestudiantes@"

# Nombres
DATABASE_NAME = "Estudiantes"
COLLECTION_NAME = "Estudiantes_Materias"
JSON_FILE = "estudiantes_documentos.json"



In [34]:
client = MongoClient(CONNECTION_STRING)

# Probar la conexión
try:
    client.server_info()
    print("✓ Conexión exitosa")
except Exception as e:
    print(f" Error: {e}")
    exit()

C:\Users\claud\AppData\Local\Temp\ipykernel_1904\4213219313.py:1: UserWarning: You appear to be connected to a CosmosDB cluster. For more information regarding feature compatibility and support please visit https://www.mongodb.com/supportability/cosmosdb
  client = MongoClient(CONNECTION_STRING)


✓ Conexión exitosa


In [35]:
db = client[DATABASE_NAME]
collection = db[COLLECTION_NAME]


In [36]:
# Leer archivo JSON

with open(JSON_FILE, 'r', encoding='utf-8') as archivo:
    documentos = json.load(archivo)

In [37]:
# Insertar documentos

for documento in tqdm(documentos, desc="Insertando documentos"):
    try:
        collection.insert_one(documento)
    except Exception as e:
      pass

Insertando documentos: 100%|██████████| 10226/10226 [18:18<00:00,  9.31it/s]


In [39]:

# Contar total
total = collection.count_documents({})
print(f"\nTotal documentos: {total}")

# Contar desertores
desertores = collection.count_documents({"estado.desertor": 1})
no_desertores = collection.count_documents({"estado.desertor": 0})

print(f"Desertores: {desertores} ({desertores/total*100:.1f}%)")
print(f"No desertores: {no_desertores} ({no_desertores/total*100:.1f}%)")



Total documentos: 10226
Desertores: 477 (4.7%)
No desertores: 9749 (95.3%)
Desertores: 477 (4.7%)
No desertores: 9749 (95.3%)


In [40]:

# Contar becados
becados = collection.count_documents({"estado.becado": "No becado"})
print(f"Becados: {becados}")

# Mostrar un ejemplo


Becados: 6864


In [41]:

print("\nDocumento de ejemplo:")
ejemplo = collection.find_one({})
print(f"  ID: {ejemplo['_id']}")
print(f"  Programa: {ejemplo['academico']['programa']}")
print(f"  Desertor: {ejemplo['estado']['desertor']}")


Documento de ejemplo:
  ID: 409967
  Programa: Derecho
  Desertor: 0


In [42]:
# Cerrar conexión
client.close()